In [1]:
import math
import random
from random import shuffle

class Domino:
    
    def buy(self,hand, n, pieces):
        i = 0
        while i<n:
            hand.append(pieces[i])
            i=i+1
        del pieces[0:n]
        return hand

    #recebe estado do jogo, retorna ações possíveis
    def possibleActions(self,state,player):
        status = state[0]
        if(player =="p1"):
            hand = state[1]
        else:
            hand = state[2]
        l_end = state[4]
        r_end = state[5]
        actions = []
        index = -1

        if(l_end==r_end==-1):
            for piece in hand:
                index +=1
                actions.append((piece[0],piece[1],"left",index))
            return actions

        for piece in hand:
            index +=1
            #peça dupla
            if(piece[0]==piece[1]):
                if(piece[0]==l_end):
                    actions.append((piece[1],"left",index))
                elif(piece[1]==r_end):
                    actions.append((piece[0],"right",index))
                continue
            if(piece[0]==l_end):
                actions.append((piece[1],"left",index))
                if(l_end==r_end): #evitar duplicacao de mesmas ações na esq e dir
                    continue
            if(piece[1]==l_end):
                actions.append((piece[0],"left",index))
                if(l_end==r_end):
                    continue
            if(piece[0]==r_end):
                actions.append((piece[1],"right",index))
                if(l_end==r_end):
                    continue
            if(piece[1]==r_end):
                actions.append((piece[0],"right",index))
                if(l_end==r_end):
                    continue
        '''
        pos 2 e 3 no actions se referem a nova ponta na mesa:
        exemplo: campo 1-5 , mão : (1,2),(5,5)
        nesse caso a peça (1,2) se encaixa no lado esquerdo, sendo
        a nova ponta esquerda o 2. A representação será: (0,1), pois
        0 é a posição da tupla na mão, e o 1 representa a posição na tupla,
        que no caso é o direito

        '''
        if not actions:
            actions.append(None)
        return actions

    def termination(self,state):#se a partida ja terminou
        status = state[0]
        p1_hand = state[1]
        p2_hand = state[2]
        l_end = state[4]
        r_end = state[5]
        if (status == 3):
            return True  
        return False

    def reward(self,state,player):#recompensa por estar nesse estado
        #em caso de vitoria
        status = state[0]

        if(status == 1):
            return 0
        if(status == 3):
            p1_hand = state[1]
            p2_hand = state[2]
            lowest_piece = 12
            for piece in p1_hand:
                value = piece[0]+piece[1]
                if(value < lowest_piece):
                    lowest_piece = value
            for piece in p2_hand:
                value = piece[0]+piece[1]
                if(value < lowest_piece):
                    state[0] = 5 #player 2 venceu tendo a menor peça no block
                    if(player == "p1"):
                        return -5 #recompensa negativa ao p1 pois perdeu
                    else:
                        return +5 #recompensa positiva ao p2 pois venceu
            state[0] = 4
            if(player == "p1"):
                return +5 #recompensa negativa ao p1 pois venceu
            else:
                return -5 #recompensa positiva ao p2 pois perdeu

            #evitar recalcular menor peça
        elif(status == 4):
            if(player == "p1"):
                return +5 #recompensa negativa ao p1 pois venceu
            else:
                return -5 #recompensa positiva ao p2 pois perdeu
        elif(status == 5):
            if(player == "p1"):
                return -5 #recompensa negativa ao p1 pois perdeu
            else:
                return +5 #recompensa positiva ao p2 pois venceu

    def startGame(self):
        status = 1 #1=in progress; 2=one player blocked;3=two players blocked;4/5=p1 won/p2 won
        pieces = [(x,y) for x in range(7) for y in range(x,7)]
        random.shuffle(pieces)
        p1_hand = self.buy([],7,pieces)
        p2_hand = self.buy([],7,pieces)
        field = []
        l_end = -1
        r_end = -1
        state = [status, p1_hand, p2_hand, field, l_end, r_end]
        return state

    def playGame(self,state,action,player):
        status = state[0]
        if(status >= 3): #jogo acabou
            return state
        if(player =="p1"):
            hand = state[1]
        else:
            hand = state[2]
            
        field = state[3]
        l_end = state[4]
        r_end = state[5]

        if(action is None):#foi bloqueado
            if(status == 2):
                state[0] = 3
            elif(status == 1):
                state[0] = 2
            return state
        
        if(status == 2): # o oponente foi bloqueado, mas eu tenho peça
            state[0] = 1
            
        if(l_end==r_end==-1):
            p_index = action[3]
            orientation = action[2]
        else:
            p_index = action[2]
            orientation = action[1]

        p = hand[p_index]
        field.append(p)
        hand.remove(p)
        if (l_end==-1 and r_end==-1):
            l_end, r_end = p
        elif (orientation == "right"):#ori e o lado desejado a manter na ponta
            r_end=action[0]
        elif (orientation == "left"):
            l_end=action[0]
                
        if(player =="p1"):
            state[1] = hand
        else:
            state[2] = hand
        state[3] = field
        state[4] = l_end
        state[5] = r_end
        return state



In [ ]:
import random
import numpy as np

class Features:
    
    def __init__(self,epsilon,beta,featureWeights):
        self.data = []
        e = epsilon # epsilon-greedy proportion
        self.beta = beta
        self.featureWeights = featureWeights
        
    def getFeaturesWeights(self):
        return self.featureWeights
    
    def featureValues(self,state,action):
        numFeatures = 20
        hand = state[1]
        l_end = state[4]
        r_end = state[5]
        pos = action[2]
        result =  np.zeros(numFeatures)
        if(l_end==r_end==-1):
            pos = action[4]
            l_end = action[0]
            r_end = action[2]
        elif(action[1]=="left"):
        #substitui uma das pontas pela nova ação
            l_end = action[0]
        else:
            r_end = action[0]
        value =hand.pop(pos)
        """
        Características a partir de informações imperfeitas.
            Somente do que é visível a um jogador comum:
                Campo, peças de suas mãos, quantidade de peças do oponente
        """
        #valida caracteristicas de acordo com o estado submetido e as retorna
        
        result[0] = self.NumDouble(hand,action,l_end,r_end,lambda x: x==1)
        result[1] = self.NumDouble(hand,action,l_end,r_end,lambda x: x==2)
        result[2] = self.NumDouble(hand,action,l_end,r_end,lambda x: x>=3)
        #impossibilita qualquer peça de ser jogada
        result[3] = self.blocksMe(hand,action,l_end,r_end)
        
        result[4] = self.NumActions(hand,action,l_end,r_end,lambda x: x==1)
        result[5] = self.NumActions(hand,action,l_end,r_end,lambda x: x==2)
        result[6] = self.NumActions(hand,action,l_end,r_end,lambda x: x>=3)
        
        for i in range(7,14):
            #quais peças duplas temos na mão, de 0:0 a 6:6
            result[i] = self.hasDouble(i-7,hand,action)
        
        """
        Características a partir de informações perfeitas.
        Sabe-se da mão do oponente, de toda e qualquer informação necessária
        para se tomar uma decisão
        """
        
        hand.insert(pos,value)
        
        return result
    
    def NumDouble(self,hand,action,l_end,r_end,exp):
        count = 0
        for piece in hand:
            if(piece[0] == piece[1]):
                count +=1
        if(exp(count)):
            return True
        return False
    
    def hasDouble(self,number,hand,action):
        for piece in hand:
            if((piece[0] == piece[1]) and (piece[0] == number)):
                return True
        return False
    
    def blocksMe(self,hand,action,l_end,r_end):      
        for piece in hand:
            if(self.isPlayable(piece,l_end,r_end)):
                #se encontra alguma peça valida na mao
                return False # então ainda não estou bloqueado
  
        return True #nao encontrou
        
    def NumActions(self,hand,action,l_end,r_end,exp):
        count = 0
        for piece in hand:
            if(self.isPlayable(piece,l_end,r_end)):
                count +=1
        if(exp(count)):#se tem mais que o numero de peças designado
            return True
        return False #se tem exatamente aquele numero
          
    def isPlayable(self,piece,l_end,r_end):#se dado a peça e as duas pontas, ela é jogavel
        if(piece[0]==l_end or piece[0]==r_end or piece[1]==l_end or piece[1]==r_end):
               return True
        return False
    

def qValue(state,act,features):
    featureVals = features.featureValues(state,act)
    val = numpy.dot (features.getFeatureWeights, featureVals)    # value before action
    return val
        
def eGreedyPicker(actions,state):
    rand = np.random.random_sample()
    bestValue = 0
    if(rand > e):
        for act in actions:
            value = qValue(state,act)
            if(value> bestValue):
                bestValue = value
        choosenAct = bestValue
    else:
        choosenAct = random.choice(actions)
        
    return choosenAct

def policyAct(state,player):
    actions = dominoes.possibleActions(state,player)
    #escolhe a partir dos valores na q-value com tecnica e-greedy
    choosenAct = eGreedyPicker(actions,state)
    
    return choosenAct


In [ ]:
def step(state,player,act,features):
    act = policyAct(state,player)                   # take action by a e-greedy policy
    val = qValue(state,act,features.getFeatureWeights())
    dominoes.playGame(state,act)                  # do selected action
    return state,val
def evaluate(state,player,val,features):
    #A recompensa é dada após a ação dos 2 players
    #não é recompensa pela ação de 1 player, mas dos dois
    r = dominoes.reward(state,player)   
    next_action = policyAct(state)                  # choose next action
    # pega o novo valor e o utiliza para dar update
    new_value = numpy.dot(features.getFeatureWeights(),features.featuresValue(state,act))
    target = r+ 0.9 * new_value                     # gamma = 0.9
    delta = target - val                            # prediction error

    #pra cada peso no array de caracteristicas aplica o gradient descent update
    #somente as caracteristicas validas no estado atual serão creditadas
    featureWeights += 0.5 * delta *features.featuresValue(state,act) 
    elements = [state,val,act]
    return elements

In [ ]:
state = dominoes.startGame() 
act = policyAct(state)         # take action by a e-greedy policy
val = qValue(state,act)
featureWeights1 = np.zeros(20)
featureWeights2 = np.zeros(20)
featuresPlayer1 = Features(0.01,1,featureWeights1)
featuresPlayer2 = Features(0.01,1,featureWeights2)
r = 0
duration = 0
while dominoes.termination() == False:
    #avaliam melhor ação(ou exploratoria) e a executam, retornando
    #o novo estado ao final do passo do ultimo player, e o valor 
    # do par estado-ação antes de executar
    state,val1 = step(state,"p1",act,featuresPlayer1)
    state,val2 = step(state,"p2",act,featuresPlayer2)

    #pegam recompensas, atualizam as características e retornam novo
    #estado, valor e proxima acao
    state,val1,act = evaluate(state,"p1",val1,featuresPlayer1)
    state,val2,act = evaluate(state,"p2",val2,featuresPlayer2)
    

    #val = new_value
    #act = next_action

